In [1]:
import numpy as np

#### 跳水板没有负载时，及 $f(x)=-480wdg$ 为常数时，有
$$
\begin{aligned}
EIy''''&=f(x)=f \\
y'''&=\frac{f}{EI}x+C_1 \\
y''&=\frac{f}{2EI}x^2+C_1x+C_2 \\
y'&=\frac{f}{6EI}x^3+\frac{C_1}{2}x^2+C_2x+C_3 \\
y&=\frac{f}{24EI}x^4+\frac{C_1}{6}x^3+\frac{C_2}{2}x^2+C_3x+C_4 \\ \\
Because \; y(0)=y'(0)&=y''(L)=y'''(L)=0\\
C_4=C_3&=0 \\
C_1&=-\frac{fL}{EI} \\
C_2&=-\frac{fL^2}{2EI}-C_1L=-\frac{fL^2}{2EI} \\ \\
Then \\
y&=\frac{f}{24EI}x^4-\frac{fL}{6EI}x^3-\frac{fL^2}{4EI}x^2 \\
 &=\frac{f}{24EI}x^2(x^2-4Lx-6L^2)
\end{aligned}
$$

In [4]:
# 检查单位匹配
L = 2 # meter
w = 0.3 # meter
d = 0.03 # meter
E = 1.3e+10 #pascal or N/m^2
I = w*d**3/12
rho = 480 # kg/m^3
f = -rho*w*d*9.81
y = lambda x: f/(24*E*I)*x**2*(x**2-4*L*x-6*L**2)
# 计算末端的偏移
print(y(2))

0.028977230769230776


In [3]:
# 2厘米偏移，看起来单位没什么问题。但是结果为正不知道对不对。TODO

In [11]:
def PA_LU_Factorization(A_:np.ndarray, printOrNot=False, check=False):
    order = A_.shape[0]

    # Upper triangular matrix
    U = A_.copy()
    # Lower triangular matrix
    L = np.eye(order)
    # Permutation matrix
    P = np.eye(order)

    # Elimination
    for col in range(order):
        # A matrix with pivot equals to 0 would be SINGULAR.
        if np.abs(A_[col][col]) < 0.00000001:
            print("Zero pivot encounterd. i=j=:", col)
            return None
        
        # Let pivot be the max element in the column.
        rowToSwap = np.argmax(U[col:, col])+col
        if rowToSwap != col:
            U[[col, rowToSwap], :] = U[[rowToSwap, col], :]
            P[[col, rowToSwap], :] = P[[rowToSwap, col], :]

        for row in range(col+1, order):
            mult = U[row][col]/U[col][col]
            U[row][col] = mult
            for k in range(col+1, order):
                U[row][k] -= mult*U[col][k]
    
    # Set L and clear U
    for row in range(1, order):
        for col in range(row):
            L[row][col] = U[row][col]
            U[row][col] = 0.

    if printOrNot:
        print('P:\n', P)
        print('L:\n', L)
        print('U:\n', U)

    if check:
        print('error:\n', np.abs(A_-np.linalg.inv(P)@L@U))

    return (P, L, U)

def TwoStepBackSubstitution(
        P:np.ndarray,
        L:np.ndarray,
        U:np.ndarray,
        b_:np.ndarray,
        printOrNot=False, check=False):
    b = b_.copy()
    order = P.shape[0]

    # PAx=Pb
    # LUx=Pb
    # Let Lc=Pb, then evaluate c
    # Let Ux=c, then evaluate x

    # Solve Lc=Pb
    b = P@b
    c = np.zeros(order)
    for row in range(order):
        for col in range(row):
            b[row] -= L[row][col]*c[col]
        c[row] = b[row]

    # Solve Ux=c
    x = np.zeros(order)
    for row in range(order)[::-1]:
        for col in range(row+1, order):
            c[row] -= U[row][col]*x[col]
        x[row] = c[row]/U[row][row]
    
    if printOrNot:
        print('x:\n', x)

    if check:
        print('error: ', np.abs(P@b_-L@U@x))

# Example 2.16
testM = np.array([[2.,1,5], [4,4,-4], [1,3,1]])
P, L, U = PA_LU_Factorization(testM, True, True)
TwoStepBackSubstitution(P, L, U, np.array([5.,0,6]), True, True)

P:
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
L:
 [[ 1.    0.    0.  ]
 [ 0.25  1.    0.  ]
 [ 0.5  -0.5   1.  ]]
U:
 [[ 4.  4. -4.]
 [ 0.  2.  2.]
 [ 0.  0.  8.]]
error:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
x:
 [-1.  2.  1.]
error:  [0. 0. 0.]


1. 写出定义(2.34)中结构矩阵A的MATLAB程序。然后，使用MATLAB的\命令或者自己设计的代码，求解系统，得到偏移 $y_i$，使用$n=10$作为网格大小.


2. 同时画出步骤1中的解和正确解，$y(x)=(f/24EI)x^2(x^2-4Lx+6L^2)$，其中f=f(x)是上面定义的常数.在横梁末端检查误差，此时 $x=Lm$，在这种简单情况下，导数近似是正确的，因而你的误差接近机器舍入的结果.

3. 返回步骤1中的计算，对于 $n=10\cdot 2^k$，其中 $k=1,\dots,11$ .对于每个n在$x=L$时的误差制表，对于哪一个n这个误差最小?为什么这个误差在某一点后随着n增加?你可以再做一张关于A的条件数的表，该条件数对应一个关于n的函数，并有助于对最后一个问题的回答. 对于一个大的k实施这个步骤，你可能需要让MATLAB把矩阵A保存为一个稀疏矩阵以避免完全消耗内存，使用命令A=sparse(n,n)初始化A，就可以得到稀疏矩阵，其他操作和前面一样，我们将在下一节中更加详细地讨论稀疏矩阵.


4. 对横梁加上正弦压力，这意味着给受力项加上一个形如 $s(x)=pg\sin{\frac{\pi}{L}x}$ 的函数。证明解
$$
y(x)=\frac{f}{24EI}x^2(x^2-4Lx+6L^2)-\frac{pgL}{EI\pi}(\frac{L^3}{\pi^3}\sin\frac{\pi}{L}x-\frac{x^3}{6}+\frac{L}{2}x^2-\frac{L^2}{\pi^2}x)
$$
$\quad$ 满足欧拉-伯努利横梁方程以及钳制-自由边界条件.

5. 重新运行第3步中对于正弦负载的计算. (确保包含了横臂自身的重量.) 设 $p=100kg/m$，同时画出你的计算结果和正确的解. 回答步骤3中的问题，并回答下面的问题: 在x=L点的误差和前面生成的$h^2$是否成比例? 你可以在1og-log图上画出误差与h，来探求该问题. 条件数起作用了吗?

6. 现在移除正弦负载并在横梁上加上70kg跳水员，在横梁的最后20cm保持平衡. 你必须对 $f(x_i)$ 在单位长度加上力一g乘上 $70/0.2kg/m，1.8\leqslant z\leqslant 2$, 使用步骤5中找到的最优的n值，并再次求解问题. 画出图并找出跳水板在自由端的偏斜度.

7. 如果我们把跳水板的自由端也固定，就得到钳制-钳制横梁，两端的边界条件相同 $y(0)=y'(0)=y'(L)=0$. 它用于对下陷结构，例如桥梁，进行建模, 从有微小差异的均匀步长格点开始 $0=x_0<x_1<\dots<x_n><x_{n+1}$，其中 $h=x_i-x_{i-1}, i=1, \dots ,n$，找出具有n个方程n个未知变量的系统，用于确定 $y_1, \dots y_n$. (这和钳制-自由版本的模型类似，除了系数矩阵A的最后两行应该是前两行的反转. )对于正弦负载进行求解，并对于横梁中心 $x=L/2$ 回答步骤5中的问题。在正弦负载下的钳制-钳制横梁的精确解是
$$
y(x)=\frac{f}{24EI}x^2(L-x)^2-\frac{pgL}{\pi^4EI}(L^3\sin\frac{\pi}{L}x+\pi x(x-L))
$$


8. 更多探索的想法:如果跳水板的宽度加倍，跳水板的偏移该如何变化? 如果跳水板的厚度加倍这种变化会更多还是更少? (两个板的质量相同.)如果横截面是圆形或者环形，但是面积和矩形截面相同，最大偏移量会如何变化?( 面积惯性模量对于半径为r的圆形截面是 $I=\pi r^4/4$，对于内半径为 $r_1$ 外半径为 $r_2$ 的环形截面的面积惯性模量是 $I=π(r_2^4-r_1^4)/4$.) 找出对于I横梁的面积惯性模量. 不同材质的杨氏模量已知并已经被制表，例如，钢材的密度是 $7850kg/m^3$ ，它对应的杨氏模量大约是 $2X10^{11}$ 帕斯卡.   欧拉-伯努利是一个相对简单、经典的模型，更多最近的模型考虑更加奇异的扭曲，例如Timoshenko 横梁，其中横梁的截面与横梁的主轴可能不垂直.